In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from typing import List, Union

# Path to your fine-tuned checkpoint directory (contains config + weights)
# Adjust if the folder is elsewhere (relative or absolute path both work)
model_path = r"checkpoint-2200"

# Load tokenizer (prefer from checkpoint; fall back to base tokenizer if needed)
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
except Exception:
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def compute_novelty_scores(
    texts: Union[str, List[str]],
    max_length: int = 512,
    batch_size: int = 16,
) -> np.ndarray:
    """Compute novelty scores for a string or list of strings using the loaded model.

    Returns a NumPy array of scores (float per input).
    """
    if isinstance(texts, str):
        texts = [texts]

    scores: List[float] = []

    for start in range(0, len(texts), batch_size):
        batch = texts[start : start + batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_length,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits  # shape: (B, 1) for regression or (B, C) for classification

        # If regression head (single value), squeeze last dim; keep batch dimension
        if logits.dim() == 2 and logits.size(-1) == 1:
            values = logits.squeeze(-1)
        else:
            # If classification head, use the logit of class 1 as a score if binary,
            # otherwise take max-logit as a generic score proxy.
            if logits.size(-1) == 2:
                values = logits[:, 1]
            else:
                values = logits.max(dim=-1).values

        scores.extend(values.detach().cpu().numpy().tolist())

    return np.array(scores)


In [20]:
# Example: provide text(s) and get novelty scores

# Single text input
# input_text = "Your text here"
# print(compute_novelty_scores(input_text))

# Multiple texts input
# input_texts = [
#     "First abstract or paragraph.",
#     "Second abstract or paragraph.",
# ]
# print(compute_novelty_scores(input_texts))



In [21]:
import re
from pathlib import Path

SECTION_HEADING_RE = re.compile(r"^([A-Z][A-Z \-/&]+):?$")


def extract_section(text: str, title: str) -> str:
    """Extract a section by its uppercase title from a plain text document.
    Matches lines like 'ABSTRACT', 'ABSTRACT AND SCOPE', optionally with a colon.
    Returns the section body until the next ALL-CAPS heading or end of text.
    """
    lines = text.splitlines()
    title_upper = title.strip().upper()

    start_idx = None
    for i, line in enumerate(lines):
        if line.strip().upper().startswith(title_upper):
            # require that it looks like a heading (mostly caps)
            start_idx = i + 1
            break
    if start_idx is None:
        return ""

    body_lines = []
    for j in range(start_idx, len(lines)):
        line = lines[j]
        # stop at next all-caps style heading
        if SECTION_HEADING_RE.match(line.strip()):
            break
        body_lines.append(line)
    return "\n".join(body_lines).strip()


# Example: read the abstract from cleaned_text.txt and score it
file_path = Path("analysis_output/cleaned_text.txt")
if file_path.exists():
    raw_text = file_path.read_text(encoding="utf-8", errors="ignore")
    abstract_text = extract_section(raw_text, "ABSTRACT")
    if not abstract_text:
        # Some docs use 'ABSTRACT AND SCOPE'
        abstract_text = extract_section(raw_text, "ABSTRACT AND SCOPE")

    if abstract_text:
        scores = compute_novelty_scores(abstract_text)
        print("Abstract novelty score:", scores[0])
    else:
        print("Could not find an ABSTRACT section in the file.")
else:
    print("File not found:", file_path)



Abstract novelty score: 0.262174516916275


In [22]:
# Quick test: score the PROBLEM STATEMENT section
from pathlib import Path

file_path = Path("analysis_output/cleaned_text.txt")
if file_path.exists():
    raw_text = file_path.read_text(encoding="utf-8", errors="ignore")
    ps_text = extract_section(raw_text, "PROBLEM STATEMENT")
    if ps_text:
        score = compute_novelty_scores(ps_text)[0]
        print("Problem Statement novelty score:", score)
    else:
        print("Could not find a PROBLEM STATEMENT section in the file.")
else:
    print("File not found:", file_path)



Problem Statement novelty score: 0.32252731919288635


In [23]:
# Explanations: Integrated Gradients over input embeddings
# If Captum is available we'll use it; otherwise fall back to a pure-PyTorch IG.
try:
    from captum.attr import IntegratedGradients  # type: ignore
except Exception:
    IntegratedGradients = None

import numpy as np
import torch
from typing import List, Tuple
from IPython.display import HTML, display


def _forward_from_embeds(input_embeds: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    outputs = model(inputs_embeds=input_embeds, attention_mask=attention_mask)
    logits = outputs.logits
    if logits.size(-1) == 1:
        return logits.squeeze(-1)
    if logits.size(-1) == 2:
        return logits[:, 1]
    return logits.max(dim=-1).values


def _ig_attribute_fallback(
    input_embeds: torch.Tensor,
    attention_mask: torch.Tensor,
    n_steps: int,
) -> torch.Tensor:
    # Simple Integrated Gradients implementation over embeddings baseline=0
    baseline = torch.zeros_like(input_embeds)
    scaled = [baseline + (float(i) / n_steps) * (input_embeds - baseline) for i in range(1, n_steps + 1)]
    total_grad = torch.zeros_like(input_embeds)

    for x in scaled:
        x = x.detach().requires_grad_(True)
        y = _forward_from_embeds(x, attention_mask)
        y = y.sum()  # batch reduce
        y.backward()
        if x.grad is None:
            continue
        total_grad = total_grad + x.grad

    avg_grad = total_grad / float(n_steps)
    attributions = (input_embeds - baseline) * avg_grad
    return attributions


def explain_with_ig(text: str, n_steps: int = 64) -> Tuple[List[str], np.ndarray, float]:
    enc = tokenizer(text, return_tensors="pt", truncation=True, padding=False, max_length=512)
    enc = {k: v.to(model.device) for k, v in enc.items()}

    with torch.no_grad():
        score_tensor = _forward_from_embeds(
            model.get_input_embeddings()(enc["input_ids"]), enc["attention_mask"]
        )
        score_val = float(score_tensor.detach().cpu().item())

    input_embeds = model.get_input_embeddings()(enc["input_ids"])  # (1, T, D)

    if IntegratedGradients is not None:
        ig = IntegratedGradients(lambda embeds: _forward_from_embeds(embeds, enc["attention_mask"]))
        attributions, _ = ig.attribute(
            inputs=input_embeds,
            baselines=torch.zeros_like(input_embeds),
            n_steps=n_steps,
            return_convergence_delta=True,
        )
    else:
        attributions = _ig_attribute_fallback(input_embeds, enc["attention_mask"], n_steps=n_steps)

    token_attr = attributions.sum(dim=-1).squeeze(0).detach().cpu().numpy()
    token_ids = enc["input_ids"].squeeze(0).tolist()
    tokens = tokenizer.convert_ids_to_tokens(token_ids)

    keep = [i for i, t in enumerate(tokens) if t not in (tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token)]
    tokens = [tokens[i] for i in keep]
    token_attr = token_attr[keep]

    merged_words: List[str] = []
    merged_scores: List[float] = []
    current_word = ""
    current_sum = 0.0

    for t, s in zip(tokens, token_attr):
        if t.startswith("##"):
            current_word = current_word + t[2:]
            current_sum += float(s)
        else:
            if current_word:
                merged_words.append(current_word)
                merged_scores.append(current_sum)
            current_word = t
            current_sum = float(s)
    if current_word:
        merged_words.append(current_word)
        merged_scores.append(current_sum)

    scores = np.array(merged_scores)
    if scores.size == 0:
        return [text], np.array([0.0]), score_val

    denom = max(1e-8, np.max(np.abs(scores)))
    scores_norm = (scores / denom).astype(float)
    return merged_words, scores_norm, score_val


def _color_for(value: float) -> str:
    v = max(-1.0, min(1.0, float(value)))
    if v >= 0:
        r = int(255 * v)
        g = int(30 * (1 - v))
        b = int(30 * (1 - v))
    else:
        v = -v
        r = int(30 * (1 - v))
        g = int(30 * (1 - v))
        b = int(255 * v)
    return f"rgb({r},{g},{b})"


def show_explanation(text: str, top_k: int = 12, n_steps: int = 64):
    words, scores, score_val = explain_with_ig(text, n_steps=n_steps)

    idx_sorted = np.argsort(scores)
    neg_idxs = idx_sorted[: top_k // 2]
    pos_idxs = idx_sorted[::-1][: top_k // 2]

    print(f"Model score: {score_val:.4f}")
    print("Top positive contributors:")
    for i in pos_idxs:
        print(f"  {words[i]}\t{scores[i]:.3f}")
    print("Top negative contributors:")
    for i in neg_idxs:
        print(f"  {words[i]}\t{scores[i]:.3f}")

    html = "<div style='line-height:2.0;'>"
    for w, s in zip(words, scores):
        color = _color_for(s)
        html += f"<span style='background:{color}; color:white; padding:2px 3px; margin:1px; border-radius:3px;'>{w}</span> "
    html += "</div>"
    display(HTML(html))

# Example: explain the Problem Statement from cleaned_text.txt
from pathlib import Path

file_path = Path("analysis_output/cleaned_text.txt")
if file_path.exists():
    raw_text = file_path.read_text(encoding="utf-8", errors="ignore")
    ps_text = extract_section(raw_text, "PROBLEM STATEMENT")
    if ps_text:
        show_explanation(ps_text)
    else:
        print("Could not find a PROBLEM STATEMENT section in the file.")
else:
    print("File not found:", file_path)



Model score: 0.3225
Top positive contributors:
  tracker	1.000
  the	0.865
  mentors	0.795
  academic	0.725
  twofold	0.687
  capstone	0.678
Top negative contributors:
  capstone	-0.972
  capstone	-0.898
  capstone	-0.808
  main	-0.776
  first	-0.720
  submitted	-0.600
